In [1]:
import pandas as pd
import numpy as np
#use sklearn.naive_bayes.MultinomialNB to predict
from sklearn.model_selection import train_test_split 
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [2]:
feedback = pd.read_csv('feedback_posts.csv')
feedback.head()

,timestamp,post_id,subject,content,response,sentiment
0,2024-05-22 10:10:47,1,React,sorry to bother you,happy to help,negative
1,2024-05-22 10:10:46,2,Git,out of left field,include me in the meeting,negative
2,2024-05-22 10:10:45,3,NPM,that being said,I do not know,positive
3,2024-05-22 10:10:44,4,Data Science,I need a partner,I do not know,positive
4,2024-05-22 10:10:43,5,Claim,a no-brainer,this is unrelated,positive


In [3]:
feedback.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  1000 non-null   object
 1   post_id    1000 non-null   int64 
 2   subject    1000 non-null   object
 3   content    1000 non-null   object
 4   response   1000 non-null   object
 5   sentiment  1000 non-null   object
dtypes: int64(1), object(5)
memory usage: 47.0+ KB


In [4]:
#Make new dataframe with just homeworld and unit type
subcont = feedback[['content', 'sentiment']].copy()
subcont

,content,sentiment
0,sorry to bother you,negative
1,out of left field,negative
2,that being said,positive
3,I need a partner,positive
4,a no-brainer,positive
...,...,...
995,sorry to bother you,negative
996,that being said,negative
997,this is exhausting,negative
998,at the end of the day,positive


In [5]:
#subcont['sentiment'] = np.where(subcont['sentiment'] == 'positive', 1, 0)
#subcont.head()

subcont[subcont[['sentiment']] == 'positive'] = 1
subcont[subcont[['sentiment']] == 'negative'] = 0

subcont.head()


,content,sentiment
0,sorry to bother you,0
1,out of left field,0
2,that being said,1
3,I need a partner,1
4,a no-brainer,1


In [6]:
subcont.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   content    1000 non-null   object
 1   sentiment  1000 non-null   object
dtypes: object(2)
memory usage: 15.8+ KB


In [7]:
subcont.astype({'sentiment': 'int32'}).dtypes

content      object
sentiment     int32
dtype: object

In [8]:
#Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(subcont['content'], subcont['sentiment'], test_size = 0.2, random_state = 42) #80% training, 20% test

In [9]:
subcont.head()

,content,sentiment
0,sorry to bother you,0
1,out of left field,0
2,that being said,1
3,I need a partner,1
4,a no-brainer,1


In [10]:
# Create a CountVectorizer to convert text data into numerical features
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [13]:
print(X_train_vec)
vectorizer.get_feature_names_out()

  (0, 40)	1
  (0, 48)	1
  (1, 57)	1
  (1, 63)	1
  (1, 10)	1
  (1, 69)	1
  (2, 31)	2
  (2, 30)	2
  (2, 65)	1
  (3, 59)	1
  (3, 35)	1
  (3, 56)	1
  (4, 4)	1
  (4, 49)	1
  (4, 39)	1
  (4, 33)	1
  (4, 50)	1
  (5, 2)	1
  (6, 44)	1
  (6, 60)	1
  (6, 55)	1
  (6, 47)	1
  (7, 41)	1
  (7, 67)	1
  (8, 2)	1
  :	:
  (792, 33)	1
  (792, 50)	1
  (793, 60)	1
  (793, 53)	1
  (793, 26)	1
  (793, 23)	1
  (794, 38)	1
  (794, 21)	1
  (795, 44)	1
  (795, 60)	1
  (795, 55)	1
  (795, 47)	1
  (796, 41)	1
  (796, 12)	1
  (797, 41)	1
  (797, 67)	1
  (798, 57)	1
  (798, 63)	1
  (798, 10)	1
  (798, 69)	1
  (799, 43)	1
  (799, 45)	1
  (799, 1)	1
  (799, 53)	1
  (799, 27)	1


array(['absolutely', 'am', 'amazing', 'answer', 'as', 'at', 'back',
       'ball', 'bang', 'being', 'bother', 'box', 'brainer', 'buck',
       'circle', 'day', 'do', 'end', 'exhausting', 'field', 'for',
       'forward', 'get', 'ground', 'hard', 'help', 'hit', 'ideas',
       'interesting', 'irregardless', 'is', 'it', 'know', 'last', 'left',
       'makes', 'meeting', 'morning', 'moving', 'my', 'need', 'no', 'not',
       'of', 'on', 'out', 'outside', 'page', 'partner', 'per', 'post',
       'provide', 'rolling', 'running', 'said', 'same', 'sense', 'sorry',
       'thanks', 'that', 'the', 'think', 'this', 'to', 'we', 'what',
       'will', 'worries', 'write', 'you', 'your'], dtype=object)

In [16]:
feature_names = vectorizer.get_feature_names_out()
X_train_vec_df = pd.DataFrame(X_train_vec.toarray(),columns = feature_names)
X_train_vec_df.head()

,absolutely,am,amazing,answer,as,at,back,ball,bang,being,...,think,this,to,we,what,will,worries,write,you,your
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Train a Multinomial Naive Bayes classifier
mnb = MultinomialNB()
mnb.fit(X_train_vec, y_train)


In [ ]:
# Create a CountVectorizer to convert text data into numerical features
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(subcont)

In [ ]:
X.toarray()

In [ ]:
vectorizer.get_feature_names_out()

In [ ]:
#Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42) #80% training, 20% test

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
# EDA 1: Distribution of Classes
class_distribution = feedback['sentiment'].value_counts()
class_distribution.plot(kind='pie', autopct='%1.1f%%', colors=['#66b3ff','#99ff99'])
plt.title('Distribution of Sentiment')
plt.show()

In [ ]:
# Generate WordCloud for Positive Content
positive_subject = ' '.join(feedback[feedback['sentiment'] == 'positive']['subject'])
positive_wordcloud = WordCloud(width=800, height=400, max_words=100, background_color='white', random_state=42).generate(positive_subject)
 
# Generate WordCloud for Negative Content
negative_subject = ' '.join(feedback[feedback['sentiment'] == 'negative']['subject'])
negative_wordcloud = WordCloud(width=800, height=400, max_words=100, background_color='white', random_state=42).generate(negative_subject)
 
# Generate WordCloud for Positive Content
positive_text = ' '.join(feedback[feedback['sentiment'] == 'positive']['content'])
positive_wordcloud2 = WordCloud(width=800, height=400, max_words=100, background_color='white', random_state=42).generate(positive_text)
 
# Generate WordCloud for Negative Content
negative_text = ' '.join(feedback[feedback['sentiment'] == 'negative']['content'])
negative_wordcloud2 = WordCloud(width=800, height=400, max_words=100, background_color='white', random_state=42).generate(negative_text)
 
# Plot the WordClouds
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.imshow(positive_wordcloud, interpolation='bilinear')
plt.title('Word Cloud for Positive Subjects')
plt.axis('off')
 
plt.subplot(1, 2, 2)
plt.imshow(negative_wordcloud, interpolation='bilinear')
plt.title('Word Cloud for Negative Subjects')
plt.axis('off') 

plt.subplot(1, 2, 1)
plt.imshow(positive_wordcloud2, interpolation='bilinear')
plt.title('Word Cloud for Positive Feedback')
plt.axis('off')
 
plt.subplot(1, 2, 2)
plt.imshow(negative_wordcloud2, interpolation='bilinear')
plt.title('Word Cloud for Negative Feedback')
plt.axis('off')
 
plt.tight_layout()
plt.show()

In [ ]:
# Create a CountVectorizer to convert text data into numerical features
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)
 
# Train a Multinomial Naive Bayes classifier
mnb = MultinomialNB(alpha=0.8, fit_prior=True, force_alpha=True)
mnb.fit(X_train_vec, y_train)

In [ ]:
print(X_train_vec.shape, len(y_train))

In [ ]:
X_train_vec.shape

In [ ]:
X_train.shape

In [ ]:
y_train

In [ ]:
X_train.head()

In [ ]:
if not all(isinstance(text, str) for text in X_train):
    raise ValueError("X_train should contain text data (strings).")